In [1]:
#

In [ ]:
# Basic data manipulation and visualisation libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['text.usetex'] = True
plt.rcParams["font.family"] = "Times"

import sys
sys.path.insert(0, "./Background_Scripts")
from Background_Scripts.ttensor import *
from Background_Scripts.utils import *
from Background_Scripts.plot_functions import *
from Background_Scripts.HCP_Data_Vis_Schaefer_100Parcels import *

#%matplotlib inline

# Magic command to load watermark
%load_ext watermark


In [3]:
# Possibility to stop warnings
import warnings

warnings.filterwarnings('ignore') 

In [ ]:
# Print versions
%watermark --author "Breno & Fernando" --date --time --python --machine --iversion --watermark --packages meshio,scikit-learn,jupyterlab,notebook

In [5]:
lineList_dir = './Schaefer_100Parcels_Atlas/Schaefer2018_100Parcels_7Networks_Tian_Subcortex_S1_3T_region_names_short.txt'
sublist_dir = './Schaefer_100Parcels_Atlas/Schaefer2018_100Parcels_7Networks_Tian_Subcortex_S1_3T_subnet_order_names.txt'
colorlist_dir = './Schaefer_100Parcels_Atlas/Schaefer2018_100Parcels_7Networks_Tian_Subcortex_S1_3T_subnet_order_colors.txt'
colornumbs_dir = './Schaefer_100Parcels_Atlas/Schaefer2018_100Parcels_7Networks_Tian_Subcortex_S1_3T_subnet_colors_number.txt'
node_coordinates_dir = './Schaefer_100Parcels_Atlas/Schaefer2018_100Parcels_7Networks_Tian_Subcortex_S1_3T_positions.txt'
hcp_data_vis_dir = './Background Scripts/HCP_Data_Vis_Schaefer_100Parcels.py'
correlation_matrix_dir = './Schaefer_100Parcels_Atlas/Averaged_Correlation_Matrix.txt'
triplet_weights_dir = './Schaefer_100Parcels_Atlas/Averaged_Zscored_Triplet_Weights.csv'

schaefer_nodes_label = np.genfromtxt(colornumbs_dir).astype(int) - 1
schaefer_nodes_color = open(colorlist_dir, "r").read().split('\n')
schaefer_nodes_subnet = open(sublist_dir, "r").read().split('\n')

In [6]:
As_ii, As_tc = get_simmetrized_ttensors(triplet_weights_dir)

In [7]:
A = np.genfromtxt(correlation_matrix_dir)
Aii_0 = np.abs(As_ii.fft[:,:,0])
Atc_0 = np.abs(As_tc.fft[:,:,0])

k_clusters = range(2,31)

sc_models_G, silhouette_score_sc_models_G, sc_embeddings_G = graph_spectral_clustering(A, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')
sc_models_Gii_0, silhouette_score_sc_models_Gii_0, sc_embeddings_Gii_0 = graph_spectral_clustering(Aii_0, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')
sc_models_Gtc_0, silhouette_score_sc_models_Gtc_0, sc_embeddings_Gtc_0 = graph_spectral_clustering(Atc_0, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=sc_embeddings_G, list_kmeans_models=sc_models_G,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}$')

plot_models_silhouette_diagrams(list_kclusters=[7], list_embeddings=sc_embeddings_Gii_0, list_kmeans_models=sc_models_Gii_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(0)}$')

plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=sc_embeddings_Gtc_0, list_kmeans_models=sc_models_Gtc_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(0)}$')

In [ ]:
plot_pie_clusters(models_list=sc_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of the brain graph $\mathcal{G}$')

plot_pie_clusters(models_list=sc_models_Gii_0, k_cluster=7, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of $\mathcal{G}_{II}^{(0)}$')

plot_pie_clusters(models_list=sc_models_Gtc_0, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of $\mathcal{G}_{TC}^{(0)}$')

In [ ]:
Plot_Brain_Clusters(models_list=sc_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}$')

Plot_Brain_Clusters(models_list=sc_models_Gii_0, k_cluster=7, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}_{II}^{(0)}$')

Plot_Brain_Clusters(models_list=sc_models_Gtc_0, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}_{TC}^{(0)}$')

In [ ]:
k_clusters = range(7,9)
layers = [58]
pre_iterations=10
iterations=5000
lamb=0.1

g = normalize_matrix(A)
danmf_models_G, silhouette_score_models_danmf_G, danmf_embeddings_G = danmf_clustering(g, k_clusters, layers=layers, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

g = normalize_matrix(Aii_0)
danmf_models_Gii_0, silhouette_score_models_danmf_Gii_0, danmf_embeddings_Gii_0 = danmf_clustering(g, k_clusters, layers=layers, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

g = normalize_matrix(Atc_0)
danmf_models_Gtc_0, silhouette_score_models_danmf_Gtc_0, danmf_embeddings_Gtc_0 = danmf_clustering(g, k_clusters, layers=layers, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=danmf_embeddings_G, list_kmeans_models=danmf_models_G,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}$')

plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=danmf_embeddings_Gii_0, list_kmeans_models=danmf_models_Gii_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(0)}$')

plot_models_silhouette_diagrams(list_kclusters=[7], list_embeddings=danmf_embeddings_Gtc_0, list_kmeans_models=danmf_models_Gtc_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(0)}$')

In [ ]:
plot_pie_clusters(models_list=danmf_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of the brain graph $\mathcal{G}$')

plot_pie_clusters(models_list=danmf_models_Gii_0, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of $\mathcal{G}_{II}^{(0)}$')

plot_pie_clusters(models_list=danmf_models_Gtc_0, k_cluster=7, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of $\mathcal{G}_{TC}^{(0)}$')

In [ ]:
Plot_Brain_Clusters(models_list=danmf_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}$')

Plot_Brain_Clusters(models_list=danmf_models_Gii_0, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}_{II}^{(0)}$')

Plot_Brain_Clusters(models_list=danmf_models_Gtc_0, k_cluster=7, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}_{TC}^{(0)}$')

In [ ]:
plot_mean_tubal_scalars(As_ii.fft,range(0,51), step_xticks=2, title=r'Mean squared root of the tubal scalar power spectral density of $\mathcal{M}_s$')

plot_mean_tubal_scalars(As_tc.fft,range(0,51), step_xticks=2, title=r'Mean squared root of the tubal scalar power spectral density of $\mathcal{T}_s$')

In [16]:
Aii_4 = np.abs(As_ii.fft[:,:,4])
Atc_4 = np.abs(As_tc.fft[:,:,4])

k_clusters = range(2,31)

sc_models_Gii_4, silhouette_score_sc_models_Gii_4, sc_embeddings_Gii_4 = graph_spectral_clustering(Aii_4, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')
sc_models_Gtc_4, silhouette_score_sc_models_Gtc_4, sc_embeddings_Gtc_4 = graph_spectral_clustering(Atc_4, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[5], list_embeddings=sc_embeddings_Gii_4, list_kmeans_models=sc_models_Gii_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(4)}$')

plot_models_silhouette_diagrams(list_kclusters=[6], list_embeddings=sc_embeddings_Gtc_4, list_kmeans_models=sc_models_Gtc_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(4)}$')

In [ ]:
plot_pie_clusters(models_list=sc_models_Gii_4, k_cluster=5, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of $\mathcal{G}_{II}^{(4)}$')

plot_pie_clusters(models_list=sc_models_Gtc_4, k_cluster=6, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the spectral clustering model of $\mathcal{G}_{TC}^{(4)}$')

In [ ]:
Plot_Brain_Clusters(models_list=sc_models_Gii_4, k_cluster=5, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$')

Plot_Brain_Clusters(models_list=sc_models_Gtc_4, k_cluster=6, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$')

In [ ]:
k_clusters = range(4,7)
layers = [58]
pre_iterations=10
iterations=5000
lamb=0.1

g = normalize_matrix(Aii_4)
danmf_models_Gii_4, silhouette_score_models_danmf_Gii_4, danmf_embeddings_Gii_4 = danmf_clustering(g, layers=layers, k_clusters=k_clusters, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

g = normalize_matrix(Atc_4)
danmf_models_Gtc_4, silhouette_score_models_danmf_Gtc_4, danmf_embeddings_Gtc_4 = danmf_clustering(g, layers=layers, k_clusters=k_clusters, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[4], list_embeddings=danmf_embeddings_Gii_4, list_kmeans_models=danmf_models_Gii_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(4)}$')

plot_models_silhouette_diagrams(list_kclusters=[5], list_embeddings=danmf_embeddings_Gtc_4, list_kmeans_models=danmf_models_Gtc_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(4)}$')

In [ ]:
plot_pie_clusters(models_list=danmf_models_Gii_4, k_cluster=4, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the DANMF clustering model of $\mathcal{G}_{II}^{(4)}$')

plot_pie_clusters(models_list=danmf_models_Gtc_4, k_cluster=5, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'Nodes distribution of the DANMF clustering model of $\mathcal{G}_{TC}^{(4)}$')

In [ ]:
Plot_Brain_Clusters(models_list=danmf_models_Gii_4, k_cluster=4, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$')

Plot_Brain_Clusters(models_list=danmf_models_Gtc_4, k_cluster=5, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$')

In [ ]:
df_cluster_labels_selected = pd.DataFrame({'subnet': schaefer_nodes_subnet,
                                           'schaefer': schaefer_nodes_label,
                                           'sc_G':get_clusters_label(sc_models_G, kcluster=8),
                                           'sc_Gii_0':get_clusters_label(sc_models_Gii_0, kcluster=7),
                                           'sc_Gtc_0':get_clusters_label(sc_models_Gtc_0, kcluster=8),
                                           'danmf_G': get_clusters_label(danmf_models_G, kcluster=8),
                                           'danmf_Gii_0': get_clusters_label(danmf_models_Gii_0, kcluster=8),
                                           'danmf_Gtc_0': get_clusters_label(danmf_models_Gtc_0, kcluster=7),
                                           'sc_Gii_4': get_clusters_label(sc_models_Gii_4, kcluster=5),
                                           'sc_Gtc_4': get_clusters_label(sc_models_Gtc_4, kcluster=6),
                                           'danmf_Gii_4': get_clusters_label(danmf_models_Gii_4, kcluster=4),
                                           'danmf_Gtc_4': get_clusters_label(danmf_models_Gtc_4, kcluster=5)})
print(df_cluster_labels_selected)

In [27]:
list_density = [0.05, 0.10, 0.15, 0.20]

for density in list_density:

    generate_binarized_adjacency_matrices(matrix=A, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_G'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_G'], matrix_name=r'$\mathcal{G}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_G_binarized_{int(density*100)}%.pdf', plot_show=False)

    generate_binarized_adjacency_matrices(matrix=Aii_0, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gii_0'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gii_0'], matrix_name=r'$\mathcal{G}_{II}^{(0)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gii_0_binarized_{int(density*100)}%.pdf', plot_show=False)

    generate_binarized_adjacency_matrices(matrix=Atc_0, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gtc_0'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gtc_0'], matrix_name=r'$\mathcal{G}_{TC}^{(0)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gtc_0_binarized_{int(density*100)}%.pdf', plot_show=False)

    generate_binarized_adjacency_matrices(matrix=Aii_4, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gii_4'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gii_4'], matrix_name=r'$\mathcal{G}_{II}^{(4)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gii_4_binarized_{int(density*100)}%.pdf', plot_show=False)

    generate_binarized_adjacency_matrices(matrix=Atc_4, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gtc_4'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gtc_4'], matrix_name=r'$\mathcal{G}_{TC}^{(4)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gtc_4_binarized_{int(density*100)}%.pdf', plot_show=False)

In [ ]:
df = pd.read_pickle('./df_peak_ii_tc_0_4_zscore_mean_tubal_scalars.pkl')
plot_correlation_scatters(df, figsize=(15,4))

In [ ]:
plot_metrics_distribution(df, figsize=(15,11))